### Restart and Run All Cells

In [1]:
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine, text

engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

year = 2025
quarter = 1
current_time = datetime.now()
formatted_time = current_time.strftime("%Y:%m:%d %H:%M:%S")
print(formatted_time)

2025:02:15 14:54:47


In [3]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit percent'.split()

format_dict = {
                'q_amt':'{:,}','q_amt_c':'{:,}','q_amt_p':'{:,}','inc_profit':'{:,}',
                'yoy_gain':'{:,}','acc_gain':'{:,}',    
                'q_eps':'{:.4f}','y_eps':'{:.4f}','aq_eps':'{:.4f}','ay_eps':'{:.4f}',
                'percent':'{:.2f}%'
              }

In [5]:
sql = """
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = 2025 AND quarter <= 1) 
OR (year = 2025-1 AND quarter >= 1+1)
ORDER BY year DESC, quarter DESC


In [7]:
dfc = pd.read_sql(sql, conlt)
dfc["Counter"] = 1
dfc_grp = dfc.groupby(["name"], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp["Counter"] == 4]
dfc_grp.head().style.format(format_dict)

,name,year,quarter,q_amt,Counter
69,FPT,8097,10,"1,447,950",4
79,GVREIT,8097,10,"463,216",4
185,TFFIF,8097,10,"-390,062",4


In [9]:
sql = """
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s-1 AND quarter <= %s) 
OR (year = %s-2 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC
"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = 2025-1 AND quarter <= 1) 
OR (year = 2025-2 AND quarter >= 1+1)
ORDER BY year DESC, quarter DESC



In [11]:
dfp = pd.read_sql(sql, conlt)
dfp["Counter"] = 1
dfp_grp = dfp.groupby(["name"], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp["Counter"] == 4]
dfp_grp.head().style.format(format_dict)

,name,year,quarter,q_amt,Counter
0,3BBIF,8093,10,"-8,619,852",4
1,ACE,8093,10,"1,052,706",4
2,ADVANC,8093,10,"30,780,229",4
3,AEONTS,8093,10,"3,165,741",4
4,AH,8093,10,"1,368,109",4


In [13]:
dfm = pd.merge(dfc_grp, dfp_grp, on="name", suffixes=(["_c", "_p"]), how="inner")
dfm["inc_profit"] = dfm["q_amt_c"] - dfm["q_amt_p"]
dfm["percent"] = round(dfm["inc_profit"] / abs(dfm["q_amt_p"]) * 100, 2)
dfm["year"] = year
dfm["quarter"] = "Q" + str(quarter)
df_percent = dfm[cols]
df_percent.head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,FPT,2025,Q1,"1,447,950","1,853,193","-405,243",-21.87%
1,GVREIT,2025,Q1,"463,216","567,474","-104,258",-18.37%
2,TFFIF,2025,Q1,"-390,062","9,461,682","-9,851,744",-104.12%


In [15]:
# Create the SQL query with parameter binding
sql = text("DELETE FROM yr_profits WHERE year = :year AND quarter = :quarter")

# Execute the query with parameters
params = {'year': year, 'quarter': f'Q{quarter}'}
rp = conlt.execute(sql, params)

# Print the number of rows affected
print("Rows deleted:", rp.rowcount)

Rows deleted: 1


In [17]:
sql = "SELECT name, id FROM tickers"
tickers = pd.read_sql(sql, conlt)
df_ins = pd.merge(df_percent, tickers, on="name", how="inner")
rcds = df_ins.values.tolist()
len(rcds)

3

In [19]:
# Convert DataFrame to list of records
rcds = df_ins.values.tolist()

# Define column names in the same order as values
columns = ['name', 'year', 'quarter', 'latest_amt', 'previous_amt', 'inc_amt', 'inc_pct', 'ticker_id']

# SQL insert statement with named parameters
sql = text("""
    INSERT INTO yr_profits 
    (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id)
    VALUES (:name, :year, :quarter, :latest_amt, :previous_amt, :inc_amt, :inc_pct, :ticker_id)
""")

try:
    # Execute inserts
    for rcd in rcds:
        # Convert list to dictionary
        params = dict(zip(columns, rcd))
        conlt.execute(sql, params)
except Exception as e:
    raise e

### End of loop

In [21]:
criteria_1 = df_ins.q_amt_c > 440_000
df_ins.loc[criteria_1, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,FPT,2025,Q1,"1,447,950","1,853,193","-405,243",-21.87%
1,GVREIT,2025,Q1,"463,216","567,474","-104,258",-18.37%


In [23]:
criteria_2 = df_ins.q_amt_p > 400_000
df_ins.loc[criteria_2, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,FPT,2025,Q1,"1,447,950","1,853,193","-405,243",-21.87%
1,GVREIT,2025,Q1,"463,216","567,474","-104,258",-18.37%
2,TFFIF,2025,Q1,"-390,062","9,461,682","-9,851,744",-104.12%


In [25]:
criteria_3 = df_ins.percent > 10.00
df_ins.loc[criteria_3, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent


In [27]:
final_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[final_criteria, cols].sort_values(by=["percent"], ascending=[False]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent


In [29]:
df_ins.loc[final_criteria, cols].sort_values(by=["name"], ascending=[True]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent


In [31]:
df_ins.loc[final_criteria, cols].sort_values(by=["name"], ascending=[True]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent


In [33]:
conlt.commit()
conlt.close()

In [35]:
current_time = datetime.now()
formatted_time = current_time.strftime("%Y:%m:%d %H:%M:%S")
print(formatted_time)

2025:02:15 14:56:38
